# Link Prediction on Heterogenous Graphs
##### Web Based Systems 2024, [FCSE](https://finki.ukim.mk/)

The topic of this exercise is link prediction on a graph generated from the extracted entities and relations from L2 - Knowledge Graph Construction.

Check the pytorch version so we can install the libraries with the adequate versions

If pytorch-geometric doesn't work, downgrade to torch==2.2.0

In [1]:
!pip install torch==2.2.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 755.5/755.5 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 44.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 67.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 45.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 MB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

In [2]:
!pip install torch_geometric

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.1/63.1 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 41.7 MB/s eta 0:00:00


In [3]:
!pip install pyg_lib torch_scatter torch_sparse torch_cluster torch_spline_conv -f https://data.pyg.org/whl/torch-2.2.0+cu121.html

Looking in links: https://data.pyg.org/whl/torch-2.2.0+cu121.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.9/10.9 MB 21.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.0/5.0 MB 25.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 943.4/943.4 kB 23.3 MB/s eta 0:00:00


In [4]:
import torch
print(torch.__version__)

2.2.0+cu121


# Link Prediction on Stock News Headlines

This colab notebook shows how to load a  `*.csv` file as input and construct a heterogeneous graph from it.
We will then use this dataset as input into a [heterogeneous graph model](https://pytorch-geometric.readthedocs.io/en/latest/notes/heterogeneous.html#hgtutorial), and use it for the task of link prediction.

We are going to use the triples extracted from the Stock News Headlines in AV 07 - Knowledge Graph Construction.

Note that we do not have many entities and relations in the dataset, since we extracted triples only from a subset of news headlines. Nevertheless, this data is enought to illustrate how to perform link prediction on heterogenous graphs.

The link prediction task then tries to predict missing industries for organizations mentioned in the stock news headlines, which is a usefull information for further tasks such as stock price prediction.

## Heterogeneous Graph Creation

First, we will import the dataset using pandas.

In [5]:
!gdown 1-6TyICt4E8JzqVpOT1RrFYfwQc1Sd0C4

Downloading...
From: https://drive.google.com/uc?id=1-6TyICt4E8JzqVpOT1RrFYfwQc1Sd0C4
To: /content/kg_stock_headlines_normalized.csv
100% 105k/105k [00:00<00:00, 61.8MB/s]


In [6]:
import pandas as pd
from torch_geometric.data import HeteroData
import torch_geometric.transforms as T

In [8]:
df = pd.read_csv("/content/kg_stock_headlines_normalized.csv")

In [9]:
df.head()

,head,head_type,type,tail,tail_type
0,Gran,org,country,Russia,loc
1,Russia,loc,item operated,Gran,org
2,Russia,loc,subsidiary,Gran,org
3,2010,time,point in time,2010,date
4,third quarter of 2010,time,point in time,2010,date


Let's see what types of entities and relations we have.

In [10]:
df['head_type'].unique()

array(['org', 'loc', 'time', 'concept', 'eve', 'media', 'per', 'misc'],
      dtype=object)

In [11]:
df['tail_type'].unique()

array(['loc', 'org', 'date', 'concept', 'eve', 'media', 'misc', 'per',
       'num', 'time'], dtype=object)

We will only use head, relation, tail triple types for which we have at least 100 extracted triples.

Note: Sometimes the extracted triples can have errors, so it's a good idea to inspect them before including them in the model, and remove any wrong information.

Before we create the heterogeneous graph, let’s take a look at the data.

In [12]:
df[['head_type', 'type', 'tail_type']].value_counts()

head_type  type                          tail_type
org        country                       loc          260
concept    subclass of                   concept      183
org        industry                      concept      122
           subsidiary                    org          102
concept    facet of                      concept       92
                                                     ... 
media      main subject                  concept        1
           measured physical quantity    concept        1
           number of parts of this work  num            1
concept    applies to jurisdiction       org            1
time       start time                    date           1
Name: count, Length: 246, dtype: int64

We can see that we have more than 100 triples only for the first 4 triple types, namely:
 * org -> country -> loc
 * concept -> subclass of -> concept
 * org -> industry -> concept
 * org -> subsidiary -> org

Let's take the top 4 types of triples to include in the graph.

In [13]:
relations = list(df[['head_type', 'type', 'tail_type']].value_counts().head(4).index)

In [14]:
relations

[('org', 'country', 'loc'),
 ('concept', 'subclass of', 'concept'),
 ('org', 'industry', 'concept'),
 ('org', 'subsidiary', 'org')]

Here, we first create a mapping that maps entry IDs to a consecutive value in the range `{ 0, ..., num_rows - 1 }`.
This is needed as we want our final data representation to be as compact as possible, *e.g.*, the representation of an organization in the first row should be accessible via `x[0]`.

Afterwards, we obtain the final `edge_index` representation of shape `[2, num_edges_of_type]` for each different type of edge.

For each triple type combination we store the nodes in a dictionary based on their type.

Let's create the edges in the COO PyTorch format.
Reminder:
 - The format looks like this: [[0, 0], [1, 2]]

The example above can be read as node 0 is connected to node 1, and node 0 is connceted to node 2.

We create a dictionary of COO indices for each head-relation-tail type.

In [15]:
id_to_node_mapping = {}

In [16]:
relations_by_type = {}

In [17]:
def get_or_set_id_node_mapping(entity, entity_type, mapping_dict):
  if entity_type not in mapping_dict:
    mapping_dict[entity_type] = {}

  id_entity = mapping_dict[entity_type].get(entity, -1)
  if id_entity == -1:
    id_entity = len(mapping_dict[entity_type])
    mapping_dict[entity_type][entity] = id_entity

  return id_entity

In [18]:
for i, row in df.iterrows():
  head = row['head']
  head_type = row['head_type']
  rel = row['type']
  tail = row['tail']
  tail_type = row['tail_type']

  if (head_type, rel, tail_type) not in relations:
    continue


  id_head = get_or_set_id_node_mapping(head, head_type, id_to_node_mapping)
  id_tail = get_or_set_id_node_mapping(tail, tail_type, id_to_node_mapping)

  rel_type = (head_type, rel, tail_type)

  if rel_type not in relations_by_type:
    relations_by_type[rel_type] = [[], []]

  relations_by_type[rel_type][0].append(id_head)
  relations_by_type[rel_type][1].append(id_tail)

In [19]:
relations_by_type.keys()

dict_keys([('org', 'country', 'loc'), ('org', 'industry', 'concept'), ('concept', 'subclass of', 'concept'), ('org', 'subsidiary', 'org')])

With this, we are ready to initialize our `HeteroData` object and pass the necessary information to it.
Note that we also pass in a `node_id` vector to each node type in order to reconstruct the original node indices from sampled subgraphs.
We also take care of adding reverse edges to the `HeteroData` object.
This allows our GNN model to use both directions of the edge for message passing:

In [21]:
data = HeteroData()

for entity_type in id_to_node_mapping.keys():
  data[entity_type].node_id = torch.arange(len(id_to_node_mapping[entity_type]))

for rel_type in relations_by_type.keys():
  data[rel_type].edge_index = torch.tensor(relations_by_type[rel_type])

# We also need to make sure to add the reverse edges from movies to users
# in order to let a GNN be able to pass messages in both directions.
# We can leverage the `T.ToUndirected()` transform for this from PyG:
data = T.ToUndirected()(data)

In [22]:
data

HeteroData(
  org={ node_id=[385] },
  loc={ node_id=[35] },
  concept={ node_id=[361] },
  (org, country, loc)={ edge_index=[2, 260] },
  (org, industry, concept)={ edge_index=[2, 122] },
  (concept, subclass of, concept)={ edge_index=[2, 352] },
  (org, subsidiary, org)={ edge_index=[2, 182] },
  (loc, rev_country, org)={ edge_index=[2, 260] },
  (concept, rev_industry, org)={ edge_index=[2, 122] }
)

We have 3 entity types:
 - 'org' - 385 nodes
 - 'loc' - 35 nodes
 - 'concept' - 361 nodes

 and 6 edge types:
 - ('org', 'country', 'loc') - 260 edges
 - (org, industry, concept) -  122 edges
 - (concept, subclass of, concept) - 352 edges
 - (org, subsidiary, org) - 182 edges


and the reverse edges


## Defining Edge-level Training Splits

Since our data is now ready-to-be-used, we can split the ratings of users into training, validation, and test splits.
This is needed in order to ensure that we leak no information about edges used during evaluation into the training phase.
For this, we make use of the [`transforms.RandomLinkSplit`](https://pytorch-geometric.readthedocs.io/en/latest/modules/transforms.html#torch_geometric.transforms.RandomLinkSplit) transformation from PyG.
This transformation randomly divides the edges in the `("org", "industry", "concept")` into training, validation and test edges.
The `disjoint_train_ratio` parameter further separates edges in the training split into edges used for message passing (`edge_index`) and edges used for supervision (`edge_label_index`).
Note that we also need to specify the reverse edge type `("concept", "rev_industry", "org")`.
This allows the `RandomLinkSplit` transform to drop reverse edges accordingly to not leak any information into the training phase.

In [23]:
target_edge_type = ("org", "industry", "concept")
target_edge_rev_type = ("concept", "rev_industry", "org")

In [24]:
# For this, we first split the set of edges into
# training (80%), validation (10%), and testing edges (10%).
# Across the training edges, we use 80% of edges for message passing,
# and 20% of edges for supervision.
# We further want to generate fixed negative edges for evaluation with a ratio of 2:1.
# Negative edges during training will be generated on-the-fly.
# We can leverage the `RandomLinkSplit()` transform for this from PyG:
transform = T.RandomLinkSplit(
    num_val=0.1,
    num_test=0.1,
    disjoint_train_ratio=0.8,
    neg_sampling_ratio=2.0,
    add_negative_train_samples=False,
    edge_types=target_edge_type,
    rev_edge_types=target_edge_rev_type,
)

train_data, val_data, test_data = transform(data)
print("Training data:")
print("==============")
print(train_data)
print()
print("Validation data:")
print("================")
print(val_data)
print("Test data:")
print("================")
print(test_data)

Training data:
HeteroData(
  org={ node_id=[385] },
  loc={ node_id=[35] },
  concept={ node_id=[361] },
  (org, country, loc)={ edge_index=[2, 260] },
  (org, industry, concept)={
    edge_index=[2, 20],
    edge_label=[78],
    edge_label_index=[2, 78],
  },
  (concept, subclass of, concept)={ edge_index=[2, 352] },
  (org, subsidiary, org)={ edge_index=[2, 182] },
  (loc, rev_country, org)={ edge_index=[2, 260] },
  (concept, rev_industry, org)={ edge_index=[2, 20] }
)

Validation data:
HeteroData(
  org={ node_id=[385] },
  loc={ node_id=[35] },
  concept={ node_id=[361] },
  (org, country, loc)={ edge_index=[2, 260] },
  (org, industry, concept)={
    edge_index=[2, 98],
    edge_label=[36],
    edge_label_index=[2, 36],
  },
  (concept, subclass of, concept)={ edge_index=[2, 352] },
  (org, subsidiary, org)={ edge_index=[2, 182] },
  (loc, rev_country, org)={ edge_index=[2, 260] },
  (concept, rev_industry, org)={ edge_index=[2, 98] }
)
Test data:
HeteroData(
  org={ node_id=[385

## Defining Mini-batch Loaders

We are now ready to create a mini-batch loader that will generate subgraphs that can be used as input into our GNN.
While this step is not strictly necessary for small-scale graphs, it is absolutely necessary to apply GNNs on larger graphs that do not fit onto GPU memory otherwise.
Here, we make use of the [`loader.LinkNeighborLoader`](https://pytorch-geometric.readthedocs.io/en/latest/modules/loader.html#torch_geometric.loader.LinkNeighborLoader) which samples multiple hops from both ends of a link and creates a subgraph from it.
Here, `edge_label_index` serves as the "seed links" to start sampling from.

In [25]:
train_data[target_edge_type]

{'edge_index': tensor([[351, 220,  34, 247,  67, 128, 287,  81, 280, 320, 343, 217, 256,  84,
           3,  48, 177,  53, 101, 212],
        [339,  34,   9, 226,  31,  80,  77,  40, 286, 303, 326,  76, 234,  45,
           3, 334, 103,  29,  74, 155]]), 'edge_label': tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1.]), 'edge_label_index': tensor([[246,  10, 280, 199, 222, 371, 209, 229, 226,  28,  56, 240,  98, 324,
           2, 372, 182, 351, 208,  48, 221, 105,  48, 246, 100,  38, 175, 199,
         262,  23, 326, 354,  67,  56, 177, 357, 115, 188, 227,  71,  74, 202,
         125,  21, 259, 381, 226, 229,  32,  30,  48, 144, 177, 231,  72,  97,
         100, 233, 211, 310,  47, 243,  6

In [26]:
from torch_geometric.loader import LinkNeighborLoader

In [27]:
# In the first hop, we sample at most 5 neighbors.
# In the second hop, we sample at most 3 neighbors.
# In addition, during training, we want to sample negative edges on-the-fly with
# a ratio of 2:1.
# We can make use of the `loader.LinkNeighborLoader` from PyG:

# Define seed edges:
edge_label_index = train_data[target_edge_type].edge_label_index
edge_label = train_data[target_edge_type].edge_label

train_loader = LinkNeighborLoader(
    data=train_data,
    num_neighbors=[10, 5],
    neg_sampling_ratio=2.0,
    edge_label_index=(target_edge_type, edge_label_index),
    edge_label=edge_label,
    batch_size=8,
    shuffle=True,
)

# Inspect a sample:
sampled_data = next(iter(train_loader))

print("Sampled mini-batch:")
print("===================")
print(sampled_data)

Sampled mini-batch:
HeteroData(
  org={
    node_id=[51],
    n_id=[51],
    num_sampled_nodes=[3],
  },
  loc={
    node_id=[8],
    n_id=[8],
    num_sampled_nodes=[3],
  },
  concept={
    node_id=[52],
    n_id=[52],
    num_sampled_nodes=[3],
  },
  (org, country, loc)={
    edge_index=[2, 28],
    e_id=[28],
    num_sampled_edges=[2],
  },
  (org, industry, concept)={
    edge_index=[2, 4],
    edge_label=[24],
    edge_label_index=[2, 24],
    e_id=[4],
    num_sampled_edges=[2],
    input_id=[8],
  },
  (concept, subclass of, concept)={
    edge_index=[2, 39],
    e_id=[39],
    num_sampled_edges=[2],
  },
  (org, subsidiary, org)={
    edge_index=[2, 18],
    e_id=[18],
    num_sampled_edges=[2],
  },
  (loc, rev_country, org)={
    edge_index=[2, 12],
    e_id=[12],
    num_sampled_edges=[2],
  },
  (concept, rev_industry, org)={
    edge_index=[2, 4],
    e_id=[4],
    num_sampled_edges=[2],
  }
)


## Creating a Heterogeneous Link-level GNN

We are now ready to create our heterogeneous GNN.
The GNN is responsible for learning enriched node representations from the surrounding subgraphs, which can be then used to derive edge-level predictions.
For defining our heterogenous GNN, we make use of [`nn.SAGEConv`](https://pytorch-geometric.readthedocs.io/en/latest/generated/torch_geometric.nn.conv.SAGEConv.html#torch_geometric.nn.conv.SAGEConv) and the [`nn.to_hetero()`](https://pytorch-geometric.readthedocs.io/en/latest/modules/nn.html#torch_geometric.nn.to_hetero_transformer.to_hetero) function, which transforms a GNN defined on homogeneous graphs to be applied on heterogeneous ones.

In addition, we define a final link-level classifier, which simply takes both node embeddings of the link we are trying to predict, and applies a dot-product on them.

As nodes do not have any node-level information, we choose to learn their features jointly via a `torch.nn.Embedding` layer. In order to improve the expressiveness each node type, we add embedding layers for each type separately.

In [34]:
from torch_geometric.nn import SAGEConv, to_hetero
import torch.nn.functional as F

class GNN(torch.nn.Module):
    def __init__(self, hidden_channels):
        super().__init__()

        self.conv1 = SAGEConv(hidden_channels, hidden_channels)  # add two layers of SageConv
        self.conv2 = SAGEConv(hidden_channels, hidden_channels)

    def forward(self, x: torch.Tensor, edge_index: torch.Tensor) -> torch.Tensor:
        x = F.relu(self.conv1(x, edge_index))  # forward pass and apply non-linearity
        x = F.relu(self.conv2(x, edge_index))
        return x

# Our final classifier applies the dot-product between source and destination
# node embeddings to derive edge-level predictions:
class Classifier(torch.nn.Module):
    def forward(self, x_head: torch.Tensor, x_tail: torch.Tensor, edge_label_index: torch.Tensor) -> torch.Tensor:
        # Convert node embeddings to edge-level representations:
        edge_feat_head = x_head[edge_label_index[0]]
        edge_feat_tail = x_tail[edge_label_index[1]]

        # Apply dot-product to get a prediction per supervision edge:
        return (edge_feat_head * edge_feat_tail).sum(dim=-1)


class Model(torch.nn.Module):
    def __init__(self, hidden_channels):
        super().__init__()
        # Since the dataset does not come with rich features, we also learn one
        # embedding matrix for each node type ('org', 'loc', and 'concept'):
        self.embeddings_layers = {}
        for entity_type in id_to_node_mapping:
          self.embeddings_layers[entity_type] = torch.nn.Embedding(data[entity_type].num_nodes, hidden_channels)

        # Instantiate homogeneous GNN:
        self.gnn = GNN(hidden_channels)

        # Convert GNN model into a heterogeneous variant:
        self.gnn = to_hetero(self.gnn, metadata=data.metadata())

        self.classifier = Classifier()

    def forward(self, data: HeteroData) -> torch.Tensor:
        x_dict = {
          entity_type: self.embeddings_layers[entity_type](data[entity_type].node_id)
          for entity_type in id_to_node_mapping
        }

        # `x_dict` holds feature matrices of all node types
        # `edge_index_dict` holds all edge indices of all edge types
        x_dict = self.gnn(x_dict, data.edge_index_dict)

        # we only try to predict one edge type, namely ('org', 'industry', 'concept')

        pred = self.classifier(
            x_dict["org"],
            x_dict["concept"],
            data[target_edge_type].edge_label_index,
        )

        return pred

Let's create the model. We will set the hidden_channels to 16 in arbitrary fashion.

In [35]:
model = Model(hidden_channels=32)

print(model)

Model(
  (gnn): GraphModule(
    (conv1): ModuleDict(
      (org__country__loc): SAGEConv(32, 32, aggr=mean)
      (org__industry__concept): SAGEConv(32, 32, aggr=mean)
      (concept__subclass_of__concept): SAGEConv(32, 32, aggr=mean)
      (org__subsidiary__org): SAGEConv(32, 32, aggr=mean)
      (loc__rev_country__org): SAGEConv(32, 32, aggr=mean)
      (concept__rev_industry__org): SAGEConv(32, 32, aggr=mean)
    )
    (conv2): ModuleDict(
      (org__country__loc): SAGEConv(32, 32, aggr=mean)
      (org__industry__concept): SAGEConv(32, 32, aggr=mean)
      (concept__subclass_of__concept): SAGEConv(32, 32, aggr=mean)
      (org__subsidiary__org): SAGEConv(32, 32, aggr=mean)
      (loc__rev_country__org): SAGEConv(32, 32, aggr=mean)
      (concept__rev_industry__org): SAGEConv(32, 32, aggr=mean)
    )
  )
  (classifier): Classifier()
)


/usr/local/lib/python3.10/dist-packages/torch_geometric/nn/to_hetero_transformer.py:165: UserWarning: The type 'subclass of' contains invalid characters which may lead to unexpected behavior. To avoid any issues, ensure that your types only contain letters, numbers and underscores.
  warnings.warn(


## Training a Heterogeneous Link-level GNN

Training our GNN is then similar to training any PyTorch model.
We move the model to the desired device, and initialize an optimizer that takes care of adjusting model parameters via stochastic gradient descent.

The training loop then iterates over our mini-batches, applies the forward computation of the model, computes the loss from ground-truth labels and obtained predictions (here we make use of binary cross entropy), and adjusts model parameters via back-propagation and stochastic gradient descent.

In [36]:
import tqdm

In [37]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Device: '{device}'")

model = model.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)
loss_fn = torch.nn.BCELoss()  # Use binary cross-entropy since we only predict the existence (non-existence) of a specific head-rel-tail type

for epoch in range(1, 201):
    total_loss = total_examples = 0
    for sampled_data in tqdm.tqdm(train_loader):
        optimizer.zero_grad()  # reset the optimizer gradients

        sampled_data.to(device)
        pred = model(sampled_data)
        pred = torch.nn.Sigmoid()(pred)  # convert the logits to normalized probabilistic output
        ground_truth = sampled_data[target_edge_type].edge_label  # get the ground truth for the target link
        loss = loss_fn(pred, ground_truth)  # calculate the loss from the predictions and ground truth

        print(pred)
        print(ground_truth)
        print('*************')

        loss.backward()
        optimizer.step()
        total_loss += float(loss) * pred.numel()
        total_examples += pred.numel()
    print(f"Epoch: {epoch:03d}, Loss: {total_loss / total_examples:.4f}")

Device: 'cpu'


100%|██████████| 10/10 [00:00<00:00, 65.63it/s]


tensor([0.9905, 0.9886, 0.9607, 0.8619, 0.9844, 0.9861, 0.9864, 0.9174, 0.9709,
        0.8603, 0.8684, 0.8715, 0.9935, 0.9998, 0.9982, 0.9871, 0.9845, 0.9678,
        0.7957, 0.9839, 0.9946, 0.8165, 0.9964, 0.9945],
       grad_fn=<SigmoidBackward0>)
tensor([1., 1., 1., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0.])
*************
tensor([0.7881, 0.9587, 0.9115, 0.8996, 0.9918, 0.8742, 0.9352, 0.9420, 0.9744,
        0.9962, 0.8297, 0.9190, 0.9775, 0.9665, 0.9966, 0.8566, 0.9758, 0.9592,
        0.9819, 0.9901, 0.9515, 0.9952, 0.9857, 0.8726],
       grad_fn=<SigmoidBackward0>)
tensor([1., 1., 1., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0.])
*************
tensor([0.8371, 0.8689, 0.9815, 0.8915, 0.9665, 0.9571, 0.8585, 0.8507, 0.8832,
        0.9674, 0.8366, 0.6982, 0.9958, 0.9894, 0.9894, 0.6538, 0.9472, 0.9261,
        0.9183, 0.9790, 0.9688, 0.9809, 0.9604, 0.9838],
       grad_fn=<SigmoidBa

  0%|          | 0/10 [00:00<?, ?it/s]

tensor([0.7463, 0.8949, 0.7406, 0.9829, 0.9078, 0.9941, 0.9858, 0.9727, 0.9943,
        0.9550, 0.9552, 0.9819, 0.9138, 0.9890, 0.9645, 0.9806, 0.9192, 0.9333,
        0.8857, 0.8527, 0.9999, 0.9997, 0.9548, 0.9368],
       grad_fn=<SigmoidBackward0>)
tensor([1., 1., 1., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0.])
*************
tensor([0.9382, 0.8407, 0.9267, 0.6657, 0.8933, 0.9791, 0.9998, 0.8016, 0.8608,
        0.9275, 0.9384, 0.9914, 0.9787, 0.9962, 0.8394, 0.9832, 0.9670, 0.9982,
        0.9554, 0.9932, 0.9177, 0.9526, 0.7312, 0.9944],
       grad_fn=<SigmoidBackward0>)
tensor([1., 1., 1., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0.])
*************
tensor([0.7868, 0.9956, 0.9697, 0.9466, 0.9220, 0.9268, 0.9831, 0.9198, 0.9809,
        0.7018, 0.9790, 0.9924, 0.9738, 0.7879, 0.8288, 0.8054, 0.9096, 0.8465,
        0.9735, 0.9527, 0.9905, 0.9394, 0.8425, 0.9252],
       grad_fn=<SigmoidBa

...

tensor([0.8629, 0.9571, 0.7716, 0.7208, 0.8174, 0.9137, 0.8105, 0.8320, 0.5000,
        0.5000, 0.5000, 0.5000, 0.5000, 0.5000, 0.5000, 0.5000, 0.5003, 0.5000,
        0.5000, 0.5296, 0.5276, 0.5000, 0.5000, 0.5000],
       grad_fn=<SigmoidBackward0>)
tensor([1., 1., 1., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0.])
*************
tensor([0.5000, 0.5000, 0.5000, 0.9388, 0.9262, 0.7709, 0.9373, 0.8366, 0.5000,
        0.5000, 0.5306, 0.5000, 0.7454, 0.5000, 0.5000, 0.5000, 0.5000, 0.5000,
        0.5000, 0.5000, 0.5000, 0.5000, 0.5000, 0.5000],
       grad_fn=<SigmoidBackward0>)
tensor([1., 1., 1., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0.])
*************
tensor([0.7678, 0.5106, 0.9643, 0.9436, 0.8675, 0.9760, 0.7408, 0.9623, 0.5551,
        0.5012, 0.5009, 0.5000, 0.5449, 0.5066, 0.5038, 0.5000, 0.5025, 0.5000,
        0.5004, 0.5160, 0.5184, 0.5000, 0.5000, 0.5000],
       grad_fn=<SigmoidBa

 50%|█████     | 5/10 [00:00<00:00, 47.17it/s]

tensor([0.7421, 0.8226, 0.5328, 0.9246, 0.9586, 0.8643, 0.9289, 0.8979, 0.5000,
        0.5000, 0.5000, 0.7268, 0.5000, 0.5082, 0.5000, 0.5000, 0.5015, 0.5085,
        0.5000, 0.5000, 0.5448, 0.5391, 0.5000, 0.5015],
       grad_fn=<SigmoidBackward0>)
tensor([1., 1., 1., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0.])
*************
tensor([0.9451, 0.8226, 0.9522, 0.8789, 0.8780, 0.6944, 0.6652, 0.5000, 0.5000,
        0.5035, 0.5070, 0.5205, 0.5000, 0.5000, 0.5000, 0.6159, 0.5000, 0.5000,
        0.5000, 0.5000, 0.5045, 0.5799, 0.6704, 0.5000],
       grad_fn=<SigmoidBackward0>)
tensor([1., 1., 1., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0.])
*************
tensor([0.9108, 0.5355, 0.7815, 0.8412, 0.7987, 0.8697, 0.8633, 0.7401, 0.5000,
        0.5585, 0.5000, 0.5117, 0.5000, 0.5000, 0.5000, 0.5057, 0.5000, 0.5000,
        0.5000, 0.5000, 0.5000, 0.6143, 0.5007, 0.5000],
       grad_fn=<SigmoidBa

100%|██████████| 10/10 [00:00<00:00, 42.96it/s]


*************
Epoch: 200, Loss: 0.5719


In [38]:
ground_truth

tensor([1., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [39]:
pred

tensor([0.9516, 0.8286, 0.8233, 0.7726, 0.5000, 0.8039, 0.5000, 0.5159, 0.5000,
        0.5648, 0.5000, 0.5170, 0.5000, 0.5000, 0.5011, 0.6280, 0.5000, 0.5000],
       grad_fn=<SigmoidBackward0>)

## Evaluating a Heterogeneous Link-level GNN

After training, we evaluate our model on useen data coming from the validation set.
For this, we define a new `LinkNeighborLoader` (which now iterates over the edges in the validation set), obtain the predictions on validation edges by running the model, and finally evaluate the performance of the model by computing the AUC score over the set of predictions and their corresponding ground-truth edges (including both positive and negative edges).

In [40]:
# Define the validation seed edges:
edge_label_index = val_data[target_edge_type].edge_label_index
edge_label = val_data[target_edge_type].edge_label

val_loader = LinkNeighborLoader(
    data=val_data,
    num_neighbors=[10, 5],
    edge_label_index=(target_edge_type, edge_label_index),
    edge_label=edge_label,
    batch_size=8,
    shuffle=False,
)

sampled_data = next(iter(val_loader))

print("Sampled mini-batch:")
print("===================")
print(sampled_data)

Sampled mini-batch:
HeteroData(
  org={
    node_id=[46],
    n_id=[46],
    num_sampled_nodes=[3],
  },
  loc={
    node_id=[8],
    n_id=[8],
    num_sampled_nodes=[3],
  },
  concept={
    node_id=[12],
    n_id=[12],
    num_sampled_nodes=[3],
  },
  (org, country, loc)={
    edge_index=[2, 25],
    e_id=[25],
    num_sampled_edges=[2],
  },
  (org, industry, concept)={
    edge_index=[2, 9],
    edge_label=[8],
    edge_label_index=[2, 8],
    e_id=[9],
    num_sampled_edges=[2],
    input_id=[8],
  },
  (concept, subclass of, concept)={
    edge_index=[2, 0],
    e_id=[0],
    num_sampled_edges=[2],
  },
  (org, subsidiary, org)={
    edge_index=[2, 21],
    e_id=[21],
    num_sampled_edges=[2],
  },
  (loc, rev_country, org)={
    edge_index=[2, 18],
    e_id=[18],
    num_sampled_edges=[2],
  },
  (concept, rev_industry, org)={
    edge_index=[2, 9],
    e_id=[9],
    num_sampled_edges=[2],
  }
)


In [41]:
from sklearn.metrics import roc_auc_score

preds = []
ground_truths = []
edges = []
for sampled_data in tqdm.tqdm(val_loader):
    with torch.no_grad():  # we set this to not calculate gradients
        edges.append(sampled_data.edge_index_dict[target_edge_type])
        sampled_data.to(device)
        out = model(sampled_data)
        p = torch.sigmoid(model(sampled_data))
        preds.append(p)
        ground_truths.append(sampled_data[target_edge_type].edge_label)

100%|██████████| 5/5 [00:00<00:00, 100.40it/s]


In [42]:
pred = torch.cat(preds, dim=0).cpu().numpy()
ground_truth = torch.cat(ground_truths, dim=0).cpu().numpy()
auc = roc_auc_score(ground_truth, pred)
print()
print(f"Validation AUC: {auc:.4f}")


Validation AUC: 0.6024


In [43]:
pred

array([0.5278429 , 0.5       , 0.50722754, 0.5       , 0.7364566 ,
       0.5141666 , 0.5       , 0.5       , 0.5       , 0.52463335,
       0.50765115, 0.9487867 , 0.60032684, 0.5       , 0.5       ,
       0.53656536, 0.5021059 , 0.5       , 0.5       , 0.54959327,
       0.5120534 , 0.5       , 0.5       , 0.5       , 0.50719196,
       0.5       , 0.5       , 0.5       , 0.97119945, 0.5       ,
       0.5       , 0.54049975, 0.5       , 0.5       , 0.5       ,
       0.51453584], dtype=float32)

In [44]:
pred.round()

array([1., 0., 1., 0., 1., 1., 0., 0., 0., 1., 1., 1., 1., 0., 0., 1., 1.,
       0., 0., 1., 1., 0., 0., 0., 1., 0., 0., 0., 1., 0., 0., 1., 0., 0.,
       0., 1.], dtype=float32)

In [45]:
ground_truth

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0.], dtype=float32)